In [1]:
!pip install torch

In [2]:
import pandas as pd                                         #Data scientist starter pack
import numpy as np
from matplotlib import pyplot as plt
import math

from collections import Counter                             #Some useful tools 
from functools import reduce

import seaborn as sns                                       #More on plotting

import torch as trc                                         #To set PyTorch framework
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, MaxPool1d, Module, Softmax, BatchNorm1d, Dropout
from torch.optim import Adam, SGD

from sklearn.model_selection import train_test_split        #Tool to fast build validation set

from datetime import datetime                               #To deal with datetime objects

import string                                               #Some useful string tools
import nltk                                                 #Natural language processing libraries
import spacy
import re                                                   #Regex tools

import random as rdm                                        #Simulation tools
import time                                                 #Ticking away the moments that make up a dull day

from tqdm import tqdm                                       #Te quiero desmasiado 

from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [3]:
dow_jones=pd.read_csv(r'C:\Users\PC\Desktop\FDS\upload_DJIA_table.csv',usecols=['Date','Open','Close'])

In [4]:
ground_truth = []
for idx in dow_jones.index:
    if dow_jones.iloc[idx].Close >= dow_jones.iloc[idx].Open:
        ground_truth.append(1)
    else:
        ground_truth.append(0)

In [5]:
dow_jones['Label'] = ground_truth


In [6]:
news = pd.read_csv(r'C:\Users\PC\Desktop\FDS\RedditNews.csv')


In [7]:
dow_jones["News"]=news["News"]

In [8]:
final_dataset = dow_jones.drop(labels=['Open','Close'], axis=1)

In [9]:
def string_to_datetime(stringed:string):
    return datetime.strptime(stringed, '%Y-%m-%d').date()
final_dataset['Date'] = final_dataset['Date'].map(lambda text:string_to_datetime(text))


In [10]:
final_dataset[:5]

,Date,Label,News
0,2016-07-01,1,A 117-year-old woman in Mexico City finally re...
1,2016-06-30,1,IMF chief backs Athens as permanent Olympic host
2,2016-06-29,1,"The president of France says if Brexit won, so..."
3,2016-06-28,1,British Man Who Must Give Police 24 Hours' Not...
4,2016-06-27,0,100+ Nobel laureates urge Greenpeace to stop o...


In [11]:
train_set = final_dataset[(final_dataset.Date >= string_to_datetime('2008-08-08')) & (final_dataset.Date <= string_to_datetime('2014-12-31'))].reset_index(drop=True)
test_set = final_dataset[(final_dataset.Date >= string_to_datetime('2015-01-02')) & (final_dataset.Date <= string_to_datetime('2016-07-01'))].reset_index(drop=True)


In [12]:
final_dataset.to_csv(r'C:\Users\PC\Desktop\FDS\engineered_datasets\final_engineered.csv', index=False)
train_set.to_csv(r'C:\Users\PC\Desktop\FDS\engineered_datasets\train_set.csv', index=False)
test_set.to_csv(r'C:\Users\PC\Desktop\FDS\engineered_datasets\test_set.csv', index=False)

In [13]:
final_dataset = pd.read_csv(r'C:\Users\PC\Desktop\FDS\engineered_datasets\final_engineered.csv')
train_set = pd.read_csv(r'C:\Users\PC\Desktop\FDS\engineered_datasets\train_set.csv')
test_set = pd.read_csv(r'C:\Users\PC\Desktop\FDS\engineered_datasets\test_set.csv')

In [14]:
train_set[:5] 

,Date,Label,News
0,2014-12-31,0,"Once Islamic State is defeated, Iraq should be..."
1,2014-12-30,0,LEGO donates 600 million DKK (90.7 million USD...
2,2014-12-29,0,Physicists have detected ripples in the fabric...
3,2014-12-26,1,Obama meets Dalai Lama in spite of China protest
4,2014-12-24,0,A chilling call: Police to contact 151 Canadia...


In [15]:
#TFIDF VECTORİZER PATH

In [16]:
tfidfvector=TfidfVectorizer()
traindataset=tfidfvector.fit_transform(train_set["News"])

In [17]:
naive=BernoulliNB()
naive.fit(traindataset,train_set["Label"])

BernoulliNB()

In [18]:
test_dataset = tfidfvector.transform(test_set["News"])
predictions = naive.predict(test_dataset)

In [19]:
predictions

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,

In [20]:
matrix=confusion_matrix(test_set["Label"],predictions)
print(matrix)
score=accuracy_score(test_set["Label"],predictions)
print(score)
report=classification_report(test_set["Label"],predictions)
print(report)

[[ 43 140]
 [ 42 153]]
0.5185185185185185
              precision    recall  f1-score   support

           0       0.51      0.23      0.32       183
           1       0.52      0.78      0.63       195

    accuracy                           0.52       378
   macro avg       0.51      0.51      0.47       378
weighted avg       0.51      0.52      0.48       378



In [21]:
#TF VECTORİZER 

In [22]:
vector= CountVectorizer()
traindataset=vector.fit_transform(train_set["News"])
naive=BernoulliNB()
naive.fit(traindataset,train_set["Label"])
test_dataset = tfidfvector.transform(test_set["News"])
predictions = naive.predict(test_dataset)

In [23]:
predictions

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,

In [24]:
matrix=confusion_matrix(test_set["Label"],predictions)
print(matrix)
score=accuracy_score(test_set["Label"],predictions)
print(score)
report=classification_report(test_set["Label"],predictions)
print(report)

[[ 43 140]
 [ 42 153]]
0.5185185185185185
              precision    recall  f1-score   support

           0       0.51      0.23      0.32       183
           1       0.52      0.78      0.63       195

    accuracy                           0.52       378
   macro avg       0.51      0.51      0.47       378
weighted avg       0.51      0.52      0.48       378

